In [3]:
import os
import sys
from pathlib import Path
#sys.path[0] = str(Path(__file__).resolve().parents[2]) # Set path for custom modules
#import warnings
#from io import StringIO

# Set path for modules
sys.path[0] = '../'

#from dotenv import load_dotenv, find_dotenv
#import numpy as np
import pandas as pd

# SQL libraries
import psycopg2
from src.pipeline.dictionaries import types_dict, replace_map
from src.pipeline.transform_data import create_full_address, split_lat_long
from src.toolkits.geospatial import geocode_from_address
from src.toolkits.postgresql import Database, Table
from src.toolkits.eda import explore_value_counts

# Set notebook display options
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# if modulename not in sys.modules: print...
#load_dotenv(find_dotenv());

In [4]:
permits = Database()
permits.list_tables()

['permits_raw']

In [5]:
#params = {"table_name":"permits_raw", "types_dict":types_dict_abbrev, "id_col":"pcis_permit_no"}
#permits.drop_table('permits_raw').drop_table('tmp_permits_raw').create_table(**params)
#!cd ../ && bash scripts/load_db.sh

In [7]:
permits_raw = Table(name="permits_raw", id_col="pcis_permit_no")
permits_raw.format_table_names(replace_map=replace_map, update=True)
permits_raw.update_types(types_dict)
data = permits_raw.fetch_data()
data = create_full_address(data)
geocode_from_address(data); # Updates dataframe in place
data = split_lat_long(data)
permits_raw.update_values(data=data, id_col="pcis_permit_no", types_dict=types_dict)

Updated names in "permits_raw".
Updated types in "permits_raw".
Cost for geocoding 19 addresses is $0.10.
Geocoding...
19 locations were assigned coordinates.
Added new columns to "permits_raw":
['latitude', 'full_address', 'longitude']
Updated types in "permits_raw".
Created temporary table "tmp_permits_raw".
Dataframe columns do not match table "permits_raw".
Rearranged dataframe columns to match "permits_raw".
Copy successful on table "permits_raw".
Updated values in "permits_raw".
